In [5]:
import pandas as pd
import numpy as np
import shutil
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Load data from CSV files
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sample_df = pd.read_csv('sample_submission.csv')

test_df.head()



,id
0,Qt1fGUB0Vz.jpeg
1,j4Rhioq7R3.jpeg
2,rD0hgFHJUZ.jpeg
3,aY5z1EJsJ6.jpeg
4,qZ3IoxD2TE.jpeg


In [6]:

# Define the directory containing the images
data_dir = 'images'
train_dir = os.path.join(data_dir, 'train_dir')

# Create train_dir if it doesn't exist
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
    os.makedirs(os.path.join(train_dir, 'IA'))
    os.makedirs(os.path.join(train_dir, 'not_IA'))

# Function to check if image file exists
def image_exists(filename):
    filepath = os.path.join(data_dir, filename)
    return os.path.exists(filepath)

# Drop rows where image files do not exist
train_df = train_df[train_df['id'].apply(image_exists)].reset_index(drop=True)
test_df = test_df[test_df['id'].apply(image_exists)].reset_index(drop=True)

# Copy images to appropriate folders
for index, row in train_df.iterrows():
    image_id = row['id']
    target = row['target']
    if target == 1:
        shutil.copy(os.path.join(data_dir, image_id), os.path.join(train_dir, 'IA'))
    else:
        shutil.copy(os.path.join(data_dir, image_id), os.path.join(train_dir, 'not_IA'))

# Step 3: Create ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Convert target column to strings
train_df['target'] = train_df['target'].astype(str)

# Step 4: Load data using ImageDataGenerator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col='id',
    y_col='target',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)
#model building  
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 6: Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10
)

# Step 7: Create validation generator
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=train_dir,  # Use the same directory as the training data
    x_col='id',
    y_col='target',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # No need to shuffle for validation
)

# Step 7 (Continued): Evaluate the model
val_loss, val_acc = model.evaluate(val_generator)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_acc)

# Step 8: Make predictions on the test set
test_generator.reset()  # Reset generator to start at the beginning
predictions = model.predict(test_generator)
predicted_classes = np.where(predictions > 0.5, 1, 0)  # Convert probabilities to binary labels

# Step 9: Save predictions to a CSV file
submission_df['target'] = predicted_classes
submission_df.to_csv('sample_submission.csv', index=False)

Found 0 validated image filenames belonging to 0 classes.


c:\Users\user\anaconda3\lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 724 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


ValueError: Must provide at least one structure